In [11]:
import warnings

warnings.filterwarnings("ignore")

In [12]:
pip install langchain openai pypdf tiktoken faiss-cpu sentence-transformers tf-keras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
# ==============================
# 운영체제(OS) 관련 기능을 사용하기 위한 모듈
# (환경변수 읽기, 파일 경로 다루기 등)
# ==============================
import os


# ==============================
# .env 파일에 저장된 환경변수를 불러오는 함수
# (API KEY 같은 민감 정보 관리용)
# ==============================
from dotenv import load_dotenv


# ==============================
# 현재 프로젝트 폴더에 있는 .env 파일을 읽어서
# 환경변수로 등록하는 함수 실행
# ==============================
load_dotenv()

# ==============================
# 환경변수 중에서 "OPENAI_API_KEY" 값을 가져와서
# 파이썬 변수 OPENAI_API_KEY 에 저장
#
# .env 파일 예시:
# OPENAI_API_KEY=sk-xxxxxxxx
# ==============================
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [14]:
# ==============================
# LangChain에서 제공하는 PDF 파일 로더 클래스 불러오기
#
# PyPDFLoader:
# PDF 파일을 읽어서
# 페이지별 텍스트 데이터로 변환해주는 도구
# ==============================
from langchain_community.document_loaders import PyPDFLoader


# ==============================
# 읽을 PDF 파일 경로 지정
#
# "./" 의미:
# 현재 파이썬 파일이 있는 폴더
#
# PyPDFLoader 객체 생성
# (아직 파일을 읽지는 않고 "읽을 준비"만 함)
# ==============================
loader=PyPDFLoader("./The_Adventures_of_Tom_Sawyer.pdf")

# ==============================
# 실제로 PDF 파일을 열어서
# 페이지별 텍스트를 읽어오는 함수 실행
#
# 반환 결과:
# document = Document 객체들의 리스트(list)
#
# 구조 예시:
# document[0] → 1페이지
# document[1] → 2페이지
# document[5] → 6페이지
# ==============================
document=loader.load()


# ==============================
# document[5]
# → PDF의 6번째 페이지 선택 (인덱스는 0부터 시작)
#
# .page_content
# → 해당 페이지의 "텍스트 내용"만 가져오기
#
# [:5000]
# → 앞에서부터 5000글자만 잘라서 출력
# (너무 길어서 터미널이 넘치는 것 방지용)
# ==============================
document[5].page_content[:5000]

'Chapter 1    The Fence \n \nTom Sawyer lived with his aunt because his mother and \nfather were dead. Tom didn’t like going to school, and he \ndidn’t like working. He liked playing and having \nadventures. One Friday, he didn’t go to school—he went \nto the river. \nAunt Polly was angry. “You’re a bad boy!” she said. \n“Tomorrow you can’t play with your friends because you \ndidn’t go to school today. Tomorrow you’re going to work \nfor me. You can paint the fence.” \nSaturday morning, Tom was not happy, but he started to \npaint the fence. His friend Jim was in the street. \nTom asked him, “Do you want to paint?” \nJim said, “No, I can’t. I’m going to get water.” \nThen Ben came to Tom’s house. He watched Tom and \nsaid, “I’m going to swim today. You can’t swim because \nyou’re working.” \nTom said, “This isn’t work. I like painting.” \n“Can I paint, too?” Ben asked. \n“No, you can’t,” Tom answered. “Aunt Polly asked me \nbecause I’m a very good painter.” \nBen said, “I’m a good pai

In [15]:
# ==============================
# LangChain에서 제공하는 FAISS 벡터 데이터베이스 불러오기
#
# FAISS:
# Facebook AI에서 만든 고속 벡터 검색 엔진
# 문서를 숫자 벡터로 저장하고
# "의미적으로 비슷한 문서"를 빠르게 찾을 수 있음
# ==============================
from langchain_classic.vectorstores import FAISS

# ==============================
# OpenAI 임베딩 모델 불러오기
#
# Embedding:
# 텍스트 → 숫자 벡터(의미 좌표) 로 변환하는 모델
#
# 예:
# "강아지" → [0.012, -0.83, 0.44, ...]
# ==============================
from langchain_classic.embeddings import OpenAIEmbeddings


# ==============================
# OpenAI 임베딩 객체 생성
#
# 내부적으로 OpenAI Embedding API 사용
# (환경변수 OPENAI_API_KEY 자동 사용)
# ==============================
embeddings=OpenAIEmbeddings()

# ==============================
# FAISS 벡터 DB 생성
#
# document:
# → PDF에서 불러온 Document 객체 리스트
#
# embeddings:
# → 텍스트를 벡터로 바꿀 방법
#
# 실행 과정:
# 1. document 안 텍스트 추출
# 2. 임베딩 생성
# 3. 벡터DB에 저장
# ==============================
db=FAISS.from_documents(document,embeddings)

In [16]:
# ==============================
# 테스트용 질문 문장 작성
#
# 실제로 검색이나 질문에 사용할 텍스트
# ==============================
text="진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"

# ==============================
# 입력 문장을 임베딩(벡터)로 변환
#
# embed_query():
# → 하나의 질문 문장을 벡터로 변환할 때 사용
#
# 결과:
# 리스트(list) 형태의 숫자 배열
# ==============================
text_embedding=embeddings.embed_query(text)


# ==============================
# 임베딩 결과 출력
#
# [:50]
# → 벡터가 너무 길어서
# 앞의 50개 숫자만 출력
# ==============================
print(text_embedding[:50])

[-0.0028138665231909497, -0.020238374776143958, -0.012738939981790699, -0.016539159741274387, -0.02259930910767389, 0.028709960370719174, -0.02787668932061184, 0.0063410627904474165, -0.012025609306402632, 0.009797242220737239, -0.015478632950228687, 0.014367605193859757, 0.0067166661065529805, -0.03338132341184305, 0.0005783971135446387, 0.02805344316490442, 0.022271049573443964, -0.00504696868553582, 0.024139597397596656, -0.027093920167150287, -0.0003393052525143871, 0.0038601892483514063, 0.01402672158245221, -0.02090751699045888, -0.0051574398382186835, 0.018306701909490415, 0.02432897625038941, -0.02041513048308164, -0.004163196670105254, -0.01867283646922085, 0.0012435932560036851, -0.0012057172991806245, -0.03547712697693411, -0.0050722191681974345, 0.001227811646132516, -0.017713313471466714, 0.01108502382139875, 0.0072658670148260844, 0.009576298984048959, 0.003251017919888031, -0.0006107494545258967, 0.003241548930682266, 0.0054288840402301934, -0.0077140655267914525, 0.0155

In [ ]:
# ==============================
# LangChain에서 HuggingFace 임베딩 모델을 사용하기 위한 클래스 불러오기
#
# HuggingFaceEmbeddings:
# HuggingFace에 공개된 문장 임베딩 모델을
# LangChain에서 쉽게 사용할 수 있도록 만든 클래스
# ==============================
from langchain_classic.embeddings import HuggingFaceEmbeddings



# ==============================
# 임베딩 모델 객체 생성
#
# model_name:
# 사용할 HuggingFace 모델 이름 지정
#
# "sentence-transformers/all-MiniLM-L6-v2"
# → 문장 의미를 벡터로 변환하는 모델
# → 빠르고 가볍고 성능도 좋아서 실무에서 많이 사용됨
#
# 실행 시:
# 1) 처음 실행 → 모델 자동 다운로드
# 2) 이후 실행 → 로컬 캐시에서 불러옴
# ==============================
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

# ==============================
# 벡터로 변환할 텍스트(문장) 준비
#
# 질문, 문서, 문단 등
# 어떤 텍스트든 임베딩 가능
# ==============================
text = "진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"


# ==============================
# 텍스트를 임베딩(숫자 벡터)로 변환
#
# embed_query():
# → 하나의 문장을 벡터로 변환할 때 사용하는 함수
#
# 내부 동작:
# 1) 문장 토큰화
# 2) 딥러닝 모델 통과
# 3) 의미 벡터 계산
# 4) 리스트(list) 형태로 반환
# ==============================
text_embedding = embeddings.embed_query(text)


# ==============================
# 임베딩 결과 출력
#
# 임베딩 벡터는 길이가 매우 김 (약 384개 숫자)
# 그래서 앞부분 50개 값만 출력
#
# [:50] 의미:
# 리스트 앞에서부터 50개 요소만 선택
# ==============================
print(text_embedding[:50])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[0.013278771191835403, 0.07225917279720306, 0.09263105690479279, -0.003979543689638376, 0.001561832963488996, -0.10306371003389359, 0.10929881781339645, 0.055662017315626144, -0.031167343258857727, -0.05020315945148468, 0.08312953263521194, -0.008924411609768867, 0.09506326168775558, -0.06980790197849274, 0.0395590178668499, -0.10899195820093155, 0.04943861812353134, 0.03736481815576553, -0.12409231066703796, -0.003315406385809183, 0.04840955510735512, -0.03108510747551918, 0.008207017555832863, 0.06326045840978622, -0.06804245710372925, -0.0102081885561347, 0.004927072208374739, -0.014940311200916767, -0.0014765590894967318, -0.006598911248147488, -0.040159497410058975, 0.0828980877995491, 0.014144640415906906, -0.011793503537774086, -0.09415137022733688, 0.002156388945877552, -0.019053105264902115, -0.03773897886276245, -0.0032710495870560408, 0.046856045722961426, -0.18111617863178253, -0.11718792468309402, 0.03504842519760132, -0.06848115473985672, 0.06553444266319275, 0.0352285616

In [ ]:
# ==============================
# LangChain에서 OpenAI의 GPT(Chat 모델)를 사용하기 위한 클래스 불러오기
#
# ChatOpenAI:
# OpenAI GPT 모델을 LangChain 환경에서
# 쉽게 사용할 수 있도록 감싸놓은(wrapper) 클래스
# ==============================
from langchain_classic.chat_models import ChatOpenAI


# ==============================
# GPT 모델 객체 생성
#
# temperature=0
# → 답변의 랜덤성 제거
# → 항상 최대한 동일하고 정확한 답변 생성
# → 문서 기반 QA(RAG)에서는 필수 설정
#
# model_name='gpt-4.1-mini'
# → 빠르고 비용 효율적인 GPT 모델
# → 검색 기반 질의응답에 충분한 성능
# ==============================
llm=ChatOpenAI(temperature=0,model_name='gpt-4.1-mini')


# ==============================
# RetrievalQA 체인 클래스 불러오기
#
# RetrievalQA:
# "문서 검색(Retrieval)" + "질문 답변(QA)"
# 기능을 하나로 묶어 자동 처리해주는 체인
# ==============================
from langchain_classic.chains import RetrievalQA




# ==============================
# FAISS 벡터 DB(db)를
# 검색 전용 객체(retriever) 형태로 변환
#
# retriever 역할:
# 질문 문장 →
# 임베딩 생성 →
# 벡터 유사도 검색 →
# 관련 문서 반환
# ==============================
retriever=db.as_retriever()


# ==============================
# RetrievalQA 체인 생성
#
# llm:
# → 답변을 생성할 GPT 모델
#
# retriever:
# → 관련 문서를 검색하는 검색기
#
# chain_type='stuff':
# → 검색된 문서를 그대로 프롬프트에
# "한 번에 붙여서(stuff)" GPT에게 전달하는 방식
# → 가장 단순하고 많이 사용하는 방법
# =========================
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever
)



# ==============================
# 사용자 질문 작성
#
# 이 질문은 GPT의 일반 지식이 아니라
# PDF 문서(벡터 DB에 저장된 내용)를
# 기반으로 답을 찾게 됨
# ==============================
query="마을 무덤에 있던 남자를 죽인 사람은 누구니?"


# ==============================
# RetrievalQA 실행
#
# 입력은 딕셔너리 형태:
# {"query": 질문문장}
#
# 내부 처리 순서:
# 1) 질문 임베딩 생성
# 2) FAISS 벡터 DB 검색
# 3) 관련 문서 추출
# 4) 문서 + 질문으로 프롬프트 생성
# 5) GPT 호출
# 6) 답변 생성
# ==============================
result=qa({"query":query})



# ==============================
# 결과 출력
#
# result 구조:
# {
#   "query": 원본 질문,
#   "result": GPT가 생성한 최종 답변,
#   "source_documents": 사용된 문서 목록
# }
#
# result['result']:
# → 최종 답변 텍스트만 출력
# ==============================
print(result['result'])

마을 무덤에 있던 남자를 죽인 사람은 인준 조(Injun Joe)입니다.
